In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_prices_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [27]:
house_prices_df.shape

(1460, 88)

In [28]:
house_prices_df.describe()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,...,mosold,yrsold,saleprice,mszoning_FV,mszoning_RH,mszoning_RL,mszoning_RM,street_Pave,totalsf,int_over_sf
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,6.321918,2007.815753,180921.195890,0.044521,0.010959,0.788356,0.149315,0.995890,2567.048630,16416.028767
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,2.703626,1.328095,79442.502883,0.206319,0.104145,0.408614,0.356521,0.063996,821.714421,8665.496074
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,1.000000,2006.000000,34900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,334.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,5.000000,2007.000000,129975.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2009.500000,10425.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,6.000000,2008.000000,163000.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2474.000000,14718.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,8.000000,2009.000000,214000.000000,0.000000,0.000000,1.000000,0.000000,1.000000,3004.000000,20105.750000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,12.000000,2010.000000,755000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11752.000000,117520.000000


In [3]:
#data cleaning
#get my categorical varibales into dummies
house_prices_df = pd.concat([house_prices_df,pd.get_dummies(house_prices_df.mszoning, prefix="mszoning", drop_first=True)], axis=1)
house_prices_df = pd.concat([house_prices_df,pd.get_dummies(house_prices_df.street, prefix="street", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(house_prices_df.mszoning, prefix="mszoning", drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(house_prices_df.street, prefix="street", drop_first=True).columns)


In [5]:
#feature engeneering
#create total sf
house_prices_df['totalsf'] = house_prices_df['totalbsmtsf'] + house_prices_df['firstflrsf'] + house_prices_df['secondflrsf']
#interaction between total sf and overall quality
house_prices_df['int_over_sf'] = house_prices_df['totalsf'] * house_prices_df['overallqual']

In [9]:
#split data into train and test

Y = house_prices_df['saleprice']
# X is the feature set
X = house_prices_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalsf', 'int_over_sf'] + dummy_column_names]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

In [17]:
lin = LinearRegression()

lin.fit(X_train, y_train)

# We are making predictions here
train_pred = lin.predict(X_train)
test_pred = lin.predict(X_test)

print('R^2 for train : {}'.format(lin.score(X_train, y_train)))
print('R^2 for test : {}'.format(lin.score(X_test, y_test)))
print('MAE for training : {}'.format(mean_absolute_error(y_train, train_pred)))
print('MAE for test : {}'.format(mean_absolute_error(y_test, test_pred)))

R^2 for train : 0.774153024324276
R^2 for test : 0.7993629437485831
MAE for training : 23105.57932975334
MAE for test : 24123.710865946305


Overall, linear regression does well on the predictions

In [23]:
#create alphas
alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]

lasso_cv = LassoCV(alphas=alphas, cv=5)

lasso_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lasso_cv.predict(X_train)
y_preds_test = lasso_cv.predict(X_test)
print(lasso_cv.alpha_)
print('R^2 for train : {}'.format(lasso_cv.score(X_train, y_train)))
print('R^2 for test : {}'.format(lasso_cv.score(X_test, y_test)))
print('MAE for training : {}'.format(mean_absolute_error(y_train, y_preds_train)))
print('MAE for test : {}'.format(mean_absolute_error(y_test, y_preds_test)))

10.0
R^2 for train : 0.7741086450380119
R^2 for test : 0.7989684137643848
MAE for training : 23105.699192036493
MAE for test : 24177.372063087365
